In [15]:
import pickle
import math
import statistics
import datetime
import os

def readPickleObject(path):
    if (os.path.exists(path) is False):
        return {}
    f = open(path, 'rb')
    result = pickle.load(f)
    f.close()
    return result

not_trading_days = set(['2018-01-01', '2018-01-06', '2018-01-07', '2018-01-13', '2018-01-14', '2018-01-15', '2018-01-20', '2018-01-21', '2018-01-27', '2018-01-28', '2018-02-03', '2018-02-04', '2018-02-10', '2018-02-11', '2018-02-17', '2018-02-18', '2018-02-19', '2018-02-24', '2018-02-25', '2018-03-03', '2018-03-04', '2018-03-10', '2018-03-11', '2018-03-17', '2018-03-18', '2018-03-24', '2018-03-25', '2018-03-30', '2018-03-31', '2018-04-01', '2018-04-07', '2018-04-08', '2018-04-14', '2018-04-15', '2018-04-21', '2018-04-22', '2018-04-28', '2018-04-29', '2018-05-05', '2018-05-06', '2018-05-12', '2018-05-13', '2018-05-19', '2018-05-20', '2018-05-26', '2018-05-27', '2018-05-28', '2018-06-02', '2018-06-03', '2018-06-09', '2018-06-10', '2018-06-16', '2018-06-17', '2018-06-23', '2018-06-24', '2018-06-30', '2018-07-01', '2018-07-04', '2018-07-07', '2018-07-08', '2018-07-14', '2018-07-15', '2018-07-21', '2018-07-22', '2018-07-28', '2018-07-29', '2018-08-04', '2018-08-05', '2018-08-11', '2018-08-12', '2018-08-18', '2018-08-19', '2018-08-25', '2018-08-26', '2018-09-01', '2018-09-02', '2018-09-03', '2018-09-08', '2018-09-09', '2018-09-15', '2018-09-16', '2018-09-22', '2018-09-23', '2018-09-29', '2018-09-30', '2018-10-06', '2018-10-07', '2018-10-13', '2018-10-14', '2018-10-20', '2018-10-21', '2018-10-27', '2018-10-28', '2018-11-03', '2018-11-04', '2018-11-10', '2018-11-11', '2018-11-17', '2018-11-18', '2018-11-22', '2018-11-24', '2018-11-25', '2018-12-01', '2018-12-02', '2018-12-05', '2018-12-08', '2018-12-09', '2018-12-15', '2018-12-16', '2018-12-22', '2018-12-23', '2018-12-25', '2018-12-29', '2018-12-30', '2019-01-01', '2019-01-05', '2019-01-06', '2019-01-12', '2019-01-13', '2019-01-19', '2019-01-20', '2019-01-21', '2019-01-26', '2019-01-27', '2019-02-02', '2019-02-03', '2019-02-09', '2019-02-10', '2019-02-16', '2019-02-17', '2019-02-18', '2019-02-23', '2019-02-24', '2019-03-02', '2019-03-03', '2019-03-09', '2019-03-10', '2019-03-16', '2019-03-17', '2019-03-23', '2019-03-24', '2019-03-30', '2019-03-31', '2019-04-06', '2019-04-07', '2019-04-13', '2019-04-14', '2019-04-19', '2019-04-20', '2019-04-21', '2019-04-27', '2019-04-28', '2019-05-04', '2019-05-05', '2019-05-11', '2019-05-12', '2019-05-18', '2019-05-19', '2019-05-25', '2019-05-26', '2019-05-27', '2019-06-01', '2019-06-02', '2019-06-08', '2019-06-09', '2019-06-15', '2019-06-16', '2019-06-22', '2019-06-23', '2019-06-29', '2019-06-30', '2019-07-04', '2019-07-06', '2019-07-07', '2019-07-13', '2019-07-14', '2019-07-20', '2019-07-21', '2019-07-27', '2019-07-28', '2019-08-03', '2019-08-04', '2019-08-10', '2019-08-11', '2019-08-17', '2019-08-18', '2019-08-24', '2019-08-25', '2019-08-31', '2019-09-01', '2019-09-02', '2019-09-07', '2019-09-08', '2019-09-14', '2019-09-15', '2019-09-21', '2019-09-22', '2019-09-28', '2019-09-29', '2019-10-05', '2019-10-06', '2019-10-12', '2019-10-13', '2019-10-19', '2019-10-20', '2019-10-26', '2019-10-27', '2019-11-02', '2019-11-03', '2019-11-09', '2019-11-10', '2019-11-16', '2019-11-17', '2019-11-23', '2019-11-24', '2019-11-28', '2019-11-30', '2019-12-01', '2019-12-07', '2019-12-08', '2019-12-14', '2019-12-15', '2019-12-21', '2019-12-22', '2019-12-25', '2019-12-28', '2019-12-29', '2020-01-01', '2020-01-04', '2020-01-05', '2020-01-11', '2020-01-12', '2020-01-18', '2020-01-19', '2020-01-20', '2020-01-25', '2020-01-26', '2020-02-01', '2020-02-02', '2020-02-08', '2020-02-09', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-22', '2020-02-23', '2020-02-29', '2020-03-01', '2020-03-07', '2020-03-08', '2020-03-14', '2020-03-15', '2020-03-21', '2020-03-22', '2020-03-28', '2020-03-29', '2020-04-04', '2020-04-05', '2020-04-10', '2020-04-11', '2020-04-12', '2020-04-18', '2020-04-19', '2020-04-25', '2020-04-26', '2020-05-02', '2020-05-03', '2020-05-09', '2020-05-10', '2020-05-16', '2020-05-17', '2020-05-23', '2020-05-24', '2020-05-25', '2020-05-30', '2020-05-31', '2020-06-06', '2020-06-07', '2020-06-13', '2020-06-14', '2020-06-20', '2020-06-21', '2020-06-27', '2020-06-28', '2020-07-03', '2020-07-04', '2020-07-05', '2020-07-11', '2020-07-12', '2020-07-18', '2020-07-19', '2020-07-25', '2020-07-26', '2020-08-01', '2020-08-02', '2020-08-08', '2020-08-09', '2020-08-15', '2020-08-16', '2020-08-22', '2020-08-23', '2020-08-29', '2020-08-30', '2020-09-05', '2020-09-06', '2020-09-07', '2020-09-12', '2020-09-13', '2020-09-19', '2020-09-20', '2020-09-26', '2020-09-27', '2020-10-03', '2020-10-04', '2020-10-10', '2020-10-11', '2020-10-17', '2020-10-18', '2020-10-24', '2020-10-25', '2020-10-31', '2020-11-01', '2020-11-07', '2020-11-08', '2020-11-14', '2020-11-15', '2020-11-21', '2020-11-22', '2020-11-26', '2020-11-28', '2020-11-29'])

trading_days = set(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05', '2018-01-08', '2018-01-09', '2018-01-10', '2018-01-11', '2018-01-12', '2018-01-16', '2018-01-17', '2018-01-18', '2018-01-19', '2018-01-22', '2018-01-23', '2018-01-24', '2018-01-25', '2018-01-26', '2018-01-29', '2018-01-30', '2018-01-31', '2018-02-01', '2018-02-02', '2018-02-05', '2018-02-06', '2018-02-07', '2018-02-08', '2018-02-09', '2018-02-12', '2018-02-13', '2018-02-14', '2018-02-15', '2018-02-16', '2018-02-20', '2018-02-21', '2018-02-22', '2018-02-23', '2018-02-26', '2018-02-27', '2018-02-28', '2018-03-01', '2018-03-02', '2018-03-05', '2018-03-06', '2018-03-07', '2018-03-08', '2018-03-09', '2018-03-12', '2018-03-13', '2018-03-14', '2018-03-15', '2018-03-16', '2018-03-19', '2018-03-20', '2018-03-21', '2018-03-22', '2018-03-23', '2018-03-26', '2018-03-27', '2018-03-28', '2018-03-29', '2018-04-02', '2018-04-03', '2018-04-04', '2018-04-05', '2018-04-06', '2018-04-09', '2018-04-10', '2018-04-11', '2018-04-12', '2018-04-13', '2018-04-16', '2018-04-17', '2018-04-18', '2018-04-19', '2018-04-20', '2018-04-23', '2018-04-24', '2018-04-25', '2018-04-26', '2018-04-27', '2018-04-30', '2018-05-01', '2018-05-02', '2018-05-03', '2018-05-04', '2018-05-07', '2018-05-08', '2018-05-09', '2018-05-10', '2018-05-11', '2018-05-14', '2018-05-15', '2018-05-16', '2018-05-17', '2018-05-18', '2018-05-21', '2018-05-22', '2018-05-23', '2018-05-24', '2018-05-25', '2018-05-29', '2018-05-30', '2018-05-31', '2018-06-01', '2018-06-04', '2018-06-05', '2018-06-06', '2018-06-07', '2018-06-08', '2018-06-11', '2018-06-12', '2018-06-13', '2018-06-14', '2018-06-15', '2018-06-18', '2018-06-19', '2018-06-20', '2018-06-21', '2018-06-22', '2018-06-25', '2018-06-26', '2018-06-27', '2018-06-28', '2018-06-29', '2018-07-02', '2018-07-03', '2018-07-05', '2018-07-06', '2018-07-09', '2018-07-10', '2018-07-11', '2018-07-12', '2018-07-13', '2018-07-16', '2018-07-17', '2018-07-18', '2018-07-19', '2018-07-20', '2018-07-23', '2018-07-24', '2018-07-25', '2018-07-26', '2018-07-27', '2018-07-30', '2018-07-31', '2018-08-01', '2018-08-02', '2018-08-03', '2018-08-06', '2018-08-07', '2018-08-08', '2018-08-09', '2018-08-10', '2018-08-13', '2018-08-14', '2018-08-15', '2018-08-16', '2018-08-17', '2018-08-20', '2018-08-21', '2018-08-22', '2018-08-23', '2018-08-24', '2018-08-27', '2018-08-28', '2018-08-29', '2018-08-30', '2018-08-31', '2018-09-04', '2018-09-05', '2018-09-06', '2018-09-07', '2018-09-10', '2018-09-11', '2018-09-12', '2018-09-13', '2018-09-14', '2018-09-17', '2018-09-18', '2018-09-19', '2018-09-20', '2018-09-21', '2018-09-24', '2018-09-25', '2018-09-26', '2018-09-27', '2018-09-28', '2018-10-01', '2018-10-02', '2018-10-03', '2018-10-04', '2018-10-05', '2018-10-08', '2018-10-09', '2018-10-10', '2018-10-11', '2018-10-12', '2018-10-15', '2018-10-16', '2018-10-17', '2018-10-18', '2018-10-19', '2018-10-22', '2018-10-23', '2018-10-24', '2018-10-25', '2018-10-26', '2018-10-29', '2018-10-30', '2018-10-31', '2018-11-01', '2018-11-02', '2018-11-05', '2018-11-06', '2018-11-07', '2018-11-08', '2018-11-09', '2018-11-12', '2018-11-13', '2018-11-14', '2018-11-15', '2018-11-16', '2018-11-19', '2018-11-20', '2018-11-21', '2018-11-23', '2018-11-26', '2018-11-27', '2018-11-28', '2018-11-29', '2018-11-30', '2018-12-03', '2018-12-04', '2018-12-06', '2018-12-07', '2018-12-10', '2018-12-11', '2018-12-12', '2018-12-13', '2018-12-14', '2018-12-17', '2018-12-18', '2018-12-19', '2018-12-20', '2018-12-21', '2018-12-24', '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-31', '2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-14', '2019-01-15', '2019-01-16', '2019-01-17', '2019-01-18', '2019-01-22', '2019-01-23', '2019-01-24', '2019-01-25', '2019-01-28', '2019-01-29', '2019-01-30', '2019-01-31', '2019-02-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-02-07', '2019-02-08', '2019-02-11', '2019-02-12', '2019-02-13', '2019-02-14', '2019-02-15', '2019-02-19', '2019-02-20', '2019-02-21', '2019-02-22', '2019-02-25', '2019-02-26', '2019-02-27', '2019-02-28', '2019-03-01', '2019-03-04', '2019-03-05', '2019-03-06', '2019-03-07', '2019-03-08', '2019-03-11', '2019-03-12', '2019-03-13', '2019-03-14', '2019-03-15', '2019-03-18', '2019-03-19', '2019-03-20', '2019-03-21', '2019-03-22', '2019-03-25', '2019-03-26', '2019-03-27', '2019-03-28', '2019-03-29', '2019-04-01', '2019-04-02', '2019-04-03', '2019-04-04', '2019-04-05', '2019-04-08', '2019-04-09', '2019-04-10', '2019-04-11', '2019-04-12', '2019-04-15', '2019-04-16', '2019-04-17', '2019-04-18', '2019-04-22', '2019-04-23', '2019-04-24', '2019-04-25', '2019-04-26', '2019-04-29', '2019-04-30', '2019-05-01', '2019-05-02', '2019-05-03', '2019-05-06', '2019-05-07', '2019-05-08', '2019-05-09', '2019-05-10', '2019-05-13', '2019-05-14', '2019-05-15', '2019-05-16', '2019-05-17', '2019-05-20', '2019-05-21', '2019-05-22', '2019-05-23', '2019-05-24', '2019-05-28', '2019-05-29', '2019-05-30', '2019-05-31', '2019-06-03', '2019-06-04', '2019-06-05', '2019-06-06', '2019-06-07', '2019-06-10', '2019-06-11', '2019-06-12', '2019-06-13', '2019-06-14', '2019-06-17', '2019-06-18', '2019-06-19', '2019-06-20', '2019-06-21', '2019-06-24', '2019-06-25', '2019-06-26', '2019-06-27', '2019-06-28', '2019-07-01', '2019-07-02', '2019-07-03', '2019-07-05', '2019-07-08', '2019-07-09', '2019-07-10', '2019-07-11', '2019-07-12', '2019-07-15', '2019-07-16', '2019-07-17', '2019-07-18', '2019-07-19', '2019-07-22', '2019-07-23', '2019-07-24', '2019-07-25', '2019-07-26', '2019-07-29', '2019-07-30', '2019-07-31', '2019-08-01', '2019-08-02', '2019-08-05', '2019-08-06', '2019-08-07', '2019-08-08', 
        '2019-08-09', '2019-08-12', '2019-08-13', '2019-08-14', '2019-08-15', '2019-08-16', '2019-08-19', '2019-08-20', '2019-08-21', '2019-08-22', '2019-08-23', '2019-08-26', '2019-08-27', '2019-08-28', '2019-08-29', '2019-08-30', '2019-09-03', '2019-09-04', '2019-09-05', '2019-09-06', '2019-09-09', '2019-09-10', '2019-09-11', '2019-09-12', '2019-09-13', '2019-09-16', '2019-09-17', '2019-09-18', '2019-09-19', '2019-09-20', '2019-09-23', '2019-09-24', '2019-09-25', '2019-09-26', '2019-09-27', '2019-09-30', '2019-10-01', '2019-10-02', '2019-10-03', '2019-10-04', '2019-10-07', '2019-10-08', '2019-10-09', '2019-10-10', '2019-10-11', '2019-10-14', '2019-10-15', '2019-10-16', '2019-10-17', '2019-10-18', '2019-10-21', '2019-10-22', '2019-10-23', '2019-10-24', '2019-10-25', '2019-10-28', '2019-10-29', '2019-10-30', '2019-10-31', '2019-11-01', '2019-11-04', '2019-11-05', '2019-11-06', '2019-11-07', '2019-11-08', '2019-11-11', '2019-11-12', '2019-11-13', '2019-11-14', '2019-11-15', '2019-11-18', '2019-11-19', '2019-11-20', '2019-11-21', '2019-11-22', '2019-11-25', '2019-11-26', '2019-11-27', '2019-11-29', '2019-12-02', '2019-12-03', '2019-12-04', '2019-12-05', '2019-12-06', '2019-12-09', '2019-12-10', '2019-12-11', '2019-12-12', '2019-12-13', '2019-12-16', '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20', '2019-12-23', '2019-12-24', '2019-12-26', '2019-12-27', '2019-12-30', '2019-12-31', '2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-06', '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-13', '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17', '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-23', '2020-04-24', '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30', '2020-05-01', '2020-05-04', '2020-05-05', '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-11', '2020-05-12', '2020-05-13', '2020-05-14', '2020-05-15', '2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21', '2020-05-22', '2020-05-26', '2020-05-27', '2020-05-28', '2020-05-29', '2020-06-01', '2020-06-02', '2020-06-03', '2020-06-04', '2020-06-05', '2020-06-08', '2020-06-09', '2020-06-10', '2020-06-11', '2020-06-12', '2020-06-15', '2020-06-16', '2020-06-17', '2020-06-18', '2020-06-19', '2020-06-22', '2020-06-23', '2020-06-24', '2020-06-25', '2020-06-26', '2020-06-29', '2020-06-30', '2020-07-01', '2020-07-02', '2020-07-06', '2020-07-07', '2020-07-08', '2020-07-09', '2020-07-10', '2020-07-13', '2020-07-14', '2020-07-15', '2020-07-16', '2020-07-17', '2020-07-20', '2020-07-21', '2020-07-22', '2020-07-23', '2020-07-24', '2020-07-27', '2020-07-28', '2020-07-29', '2020-07-30', '2020-07-31', '2020-08-03', '2020-08-04', '2020-08-05', '2020-08-06', '2020-08-07', '2020-08-10', '2020-08-11', '2020-08-12', '2020-08-13', '2020-08-14', '2020-08-17', '2020-08-18', '2020-08-19', '2020-08-20', '2020-08-21', '2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28', '2020-08-31', '2020-09-01', '2020-09-02', '2020-09-03', '2020-09-04', '2020-09-08', '2020-09-09', '2020-09-10', '2020-09-11', '2020-09-14', '2020-09-15', '2020-09-16', '2020-09-17', '2020-09-18', '2020-09-21', '2020-09-22', '2020-09-23', '2020-09-24', '2020-09-25', '2020-09-28', '2020-09-29', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-05', '2020-10-06', '2020-10-07', '2020-10-08', '2020-10-09', '2020-10-12', '2020-10-13', '2020-10-14', '2020-10-15', '2020-10-16', '2020-10-19', '2020-10-20', '2020-10-21', '2020-10-22', '2020-10-23', '2020-10-26', '2020-10-27', '2020-10-28', '2020-10-29', '2020-10-30', '2020-11-02', '2020-11-03', '2020-11-04', '2020-11-05', '2020-11-06', '2020-11-09', '2020-11-10', '2020-11-11', '2020-11-12', '2020-11-13', '2020-11-16', '2020-11-17', '2020-11-18', '2020-11-19', '2020-11-20', '2020-11-23', '2020-11-24', '2020-11-25', '2020-11-27', '2020-11-30', '2020-12-01', '2020-12-02', '2020-12-03', '2020-12-04', '2020-12-07', '2020-12-08', '2020-12-09', '2020-12-10', '2020-12-11', '2020-12-14', '2020-12-15', '2020-12-16', '2020-12-17', '2020-12-18', '2020-12-21', '2020-12-22', '2020-12-23', '2020-12-24', '2020-12-28', '2020-12-29', '2020-12-30', '2020-12-31', '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08', '2021-01-11', '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15', '2021-01-19', '2021-01-20', '2021-01-21', '2021-01-22', '2021-01-25', '2021-01-26', '2021-01-27', '2021-01-28', '2021-01-29', '2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05', '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11', '2021-02-12', '2021-02-16', '2021-02-17', '2021-02-18', '2021-02-19', '2021-02-22', '2021-02-23', '2021-02-24', '2021-02-25', '2021-02-26', '2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05', '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11', '2021-03-12', '2021-03-15', '2021-03-16', '2021-03-17', 
        '2021-03-18', '2021-03-19', '2021-03-22', '2021-03-23', '2021-03-24', '2021-03-25', '2021-03-26', '2021-03-29', '2021-03-30', '2021-03-31', '2021-04-01', '2021-04-05', '2021-04-06', '2021-04-07', '2021-04-08', '2021-04-09', '2021-04-12', '2021-04-13', '2021-04-14', '2021-04-15', '2021-04-16', '2021-04-19', '2021-04-20', '2021-04-21', '2021-04-22', '2021-04-23', '2021-04-26', '2021-04-27', '2021-04-28', '2021-04-29', '2021-04-30', '2021-05-03', '2021-05-04', '2021-05-05', '2021-05-06', '2021-05-07', '2021-05-10', '2021-05-11', '2021-05-12', '2021-05-13', '2021-05-14', '2021-05-17', '2021-05-18', '2021-05-19', '2021-05-20', '2021-05-21', '2021-05-24', '2021-05-25', '2021-05-26', '2021-05-27', '2021-05-28', '2021-06-01', '2021-06-02', '2021-06-03', '2021-06-04', '2021-06-07', '2021-06-08', '2021-06-09', '2021-06-10', '2021-06-11', '2021-06-14', '2021-06-15', '2021-06-16', '2021-06-17', '2021-06-18', '2021-06-21', '2021-06-22', '2021-06-23', '2021-06-24', '2021-06-25', '2021-06-28', '2021-06-29', '2021-06-30', '2021-07-01', '2021-07-02', '2021-07-06', '2021-07-07', '2021-07-08', '2021-07-09', '2021-07-12', '2021-07-13', '2021-07-14', '2021-07-15', '2021-07-16', '2021-07-19', '2021-07-20', '2021-07-21', '2021-07-22', '2021-07-23', '2021-07-26', '2021-07-27', '2021-07-28', '2021-07-29', '2021-07-30', '2021-08-02', '2021-08-03', '2021-08-04', '2021-08-05', '2021-08-06', '2021-08-09', '2021-08-10', '2021-08-11', '2021-08-12', '2021-08-13', '2021-08-16', '2021-08-17', '2021-08-18', '2021-08-19', '2021-08-20', '2021-08-23', '2021-08-24', '2021-08-25', '2021-08-26', '2021-08-27', '2021-08-30', '2021-08-31', '2021-09-01', '2021-09-02', '2021-09-03', '2021-09-07', '2021-09-08', '2021-09-09', '2021-09-10', '2021-09-13', '2021-09-14', '2021-09-15', '2021-09-16', '2021-09-17', '2021-09-20', '2021-09-21', '2021-09-22', '2021-09-23', '2021-09-24', '2021-09-27', '2021-09-28', '2021-09-29', '2021-09-30', '2021-10-01', '2021-10-04', '2021-10-05', '2021-10-06', '2021-10-07', '2021-10-08', '2021-10-11', '2021-10-12', '2021-10-13', '2021-10-14', '2021-10-15', '2021-10-18', '2021-10-19', '2021-10-20', '2021-10-21', '2021-10-22', '2021-10-25', '2021-10-26', '2021-10-27', '2021-10-28', '2021-10-29', '2021-11-01', '2021-11-02', '2021-11-03', '2021-11-04', '2021-11-05', '2021-11-08', '2021-11-09', '2021-11-10', '2021-11-11', '2021-11-12', '2021-11-15', '2021-11-16', '2021-11-17', '2021-11-18', '2021-11-19', '2021-11-22', '2021-11-23', '2021-11-24', '2021-11-26', '2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03', '2021-12-06', '2021-12-07', '2021-12-08', '2021-12-09', '2021-12-10', '2021-12-13', '2021-12-14', '2021-12-15', '2021-12-16', '2021-12-17', '2021-12-20', '2021-12-21', '2021-12-22', '2021-12-23', '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31'])
    

def isTradingDay(time):
    return '%d-%02d-%02d' % (time.year, time.month, time.day) not in not_trading_days

def findCloseOpenCached(symbol, time, cached_prices):
    day_increment = datetime.timedelta(days=1)

    # Find first day if tweeted after 4pm
    # If 4:00 on Wed, first day is Thursday
    # If 4:00 on Friday, first day is Monday
    if (time.hour >= 16):
        time += day_increment

    # If saturday, sunday or holiday, find first trading day to start from time
    while (isTradingDay(time) == False):
        time += day_increment

    # Find next day based on the picked first day
    end_day = time + day_increment
    while (isTradingDay(end_day) == False):
        end_day += day_increment

    start_str = '%d-%02d-%02d' % (time.year, time.month, time.day)
    end_str = '%d-%02d-%02d' % (end_day.year, end_day.month, end_day.day)
    if (start_str not in cached_prices or end_str not in cached_prices or 
        symbol not in cached_prices[start_str] or 
        symbol not in cached_prices[end_str]):
        return None

    start = cached_prices[start_str][symbol]
    end = cached_prices[end_str][symbol]
    closePrice = start[1]
    openPrice = end[0]
    return (closePrice, openPrice, (openPrice - closePrice) * 100 / closePrice)

def getTopStockDailyCached(date, num, cached_stockcounts):
    bad_stocks = set(['JCP', 'INPX', 'LK', 'HTZ', None, 'SPEX', 'NNVC', 'HSGX', 'LGCY', 
        'YRIV', 'MLNT', 'IFRX', 'OBLN', 'MLNT', 'MDR', 'FLKS', 'RTTR', 'CORV', 'WORX', 'BRK.B', 'PMTS',
        'HAIR', 'I', 'FTSV', 'DWT', 'ADOM', 'JMU', 'JASN', 'YRIV', 'LLEX', 'MDCO', 'CYTR', 'ACHN', 
        'GEMP', 'VICL', 'IGLD', 'MLNT', 'MDXG', 'TRNX', 'STML', 'ATIS', 'INSY', 'CY', 'DERM', 'DYN', 
        'ZFGN', 'BURG', 'NVTR', 'BTX', 'AKRX', 'LRAD', 'GTXI', 'LINK', 'TSG', 'VIAB', 'XGTI', 'ARRY', 
        'RARX', 'AGN', 'GBTC', 'RVLT', '', 'SNNA', 'BRK', 'SPHS', 'RBZ', 'HMNY', 'AOBC', 'TTS', 
        'HK', 'HPJ', 'GNMX', 'SORL', 'UPL', 'SYMC', 'GNC', 'STUDY', 'FLKS', 'RTTR', 'MNI', 'SSC', 
        'SPX', 'DRYS', 'SAEX', 'INNT', 'ATAI', 'CUI', 'XOG', 'MICR', 'NLNK', 'KMD', 'ROX', 'ABIL', 
        'AWSM', 'PMTS', 'CLDA', 'DO', 'EMES', 'ARCI', 'GASL', 'OILU', 'ROAN', 'CLD', 'FTNW', 'ESTR', 
        'CRZO', 'HEB', 'AYR', 'DEAC', 'PGNX', 'DOVA', 'DCAR', 'SBOT', 'PYX', 'PRTO', 'HSGX', 'OHRP', 
        'FOMX', 'CUR', 'TVIA', 'THOR', 'AMR', 'HCLP', 'CLC', 'VTIQ', 'FREEZ', 'RTN', 'PNRL', 'XON', 
        'FALC', 'PVTL', 'SSI', 'SES', 'KMPH', 'DJIA', 'PTX', 'BKS', 'NTGN', 'SPEX', 'AETI', 'DATA', 
        'DF', 'UNT', 'CTRV', 'CORV', 'TUES', 'BPMX', 'DCIX', 'TROV', 'GST', 'LGCY', 'ARQL', 'BTC', 
        'HIIQ', 'HLTH', 'UTX', 'UWT', 'LTM', 'MTFB', 'KOOL', 'CYTX', 'MYND', 'CNAT', 'SDRL', 'OASM', 
        'CVRS', 'PIR', 'CTST', 'CTRP', 'YUMA', 'FRED', 'KEG', 'MDR', 'XRP', 'CIFS', 'PTIE', 'P', 'DEST', 
        'RGSE', 'IPCI', 'CHK', 'BAS', 'AREX', 'PES', 'YOGA', 'AKS', 'VLRX', 'VIA', 'TOCA', 'S', 'TTNP',
        'DAX', 'MTNB', 'WORX'])
    datestring_id = date.strftime("%Y%m%d")

    stock_list = []
    if (datestring_id in cached_stockcounts):
        stock_list = cached_stockcounts[datestring_id]
    else:
        return None

    stock_list = sorted(stock_list, key=lambda k: k['count'], reverse=True)
    stock_list = list(filter(lambda document: document['_id'] not in bad_stocks, stock_list))
    newlist = list(map(lambda document: document['_id'], stock_list))
    result = newlist[:num]
    return result

def findTradingDays(date, upToDate):
    delta = datetime.timedelta(1)
    dates = []
    while (date <= upToDate):
        if (date.strftime("%Y-%m-%d") in trading_days):
            dates.append(date)
        date += delta
    return dates

# Find all tweets on this given day from database
def findTweets(symbol, date, tweets_per_stock):
    # Find start end and end dates for the given date
    day_increment = datetime.timedelta(days=1)
    date_end = datetime.datetime(date.year, date.month, date.day, 16)
    date_start = date_end - day_increment
    dates = [date_end, date_start]
    while (isTradingDay(date_start) == False):
        date_start -= day_increment
        dates.append(date_start)

    tweets = []
    for date in dates:
        date_string = date.strftime("%Y-%m-%d")
        if (date_string not in tweets_per_stock):
            continue
        found_tweets = tweets_per_stock[date_string]
        tweets.extend(found_tweets)

    start_index = len(tweets) - 1
    end_index = 0
    found_start = False
    found_end = False
    for i in range(len(tweets)):
        tweet_time = tweets[i]['time']
        if (found_end == False and tweet_time <= date_end):
            end_index = i
            found_end = True

        if (found_start == False and tweet_time <= date_start):
            start_index = i
            found_start = True

    return tweets[end_index:start_index]



In [26]:
close_open_prices = readPickleObject('stockCloseOpens.pkl')
user_features = readPickleObject('user_features.pickle')
daily_stock_counts = readPickleObject('newPickled/daily_stocks_cached.pickle')

In [29]:
# Get top n stocks for a datetime
start_date = datetime.datetime(2020, 6, 13)
top_stocks = getTopStockDailyCached(start_date, 10, daily_stock_counts)

# Find all trading days within range
trading_dates = findTradingDays(start_date, datetime.datetime(2020, 7, 13))

# Find all tweets for a trading day
symbol = 'AAPL'
stock_path = 'stock_files/' + symbol + '.pkl'
tweets_per_stock = readPickleObject(stock_path)
tweets = findTweets(symbol, start_date, tweets_per_stock)

# Find close open stock price
close_open = findCloseOpenCached(symbol, start_date, close_open_prices)

# A user's features
username = '10diamonds'
found_user_features = user_features[username]